In [252]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import normalize
import sklearn.linear_model as linreg
from sklearn.metrics import accuracy_score
import datetime
import ast
import statsmodels.formula.api as smf
from sklearn.linear_model import Lasso, Ridge
warnings.filterwarnings("ignore")
# %matplotlib inline

In [253]:
from sklearn.model_selection import train_test_split, GridSearchCV
#from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from textwrap import wrap

In [254]:
zipcode_neighborhood_df_test_exc_income = pd.read_csv('zipcode_neighborhood_df_test_exc_income.csv', index_col = 0)

In [255]:
dataset = zipcode_neighborhood_df_test_exc_income
target = zipcode_neighborhood_df_test_exc_income['gentrification_label']
dataset = dataset.replace('-', 0)

In [256]:
dataset=dataset.reset_index()
dataset=dataset.iloc[:,3:]
dataset

,gentrification_label,avg_rating,avg_price,starbucks_count,bars,"%, Less than 9th grade","%, 9th to 12th grade, no diploma",% High school graduate (includes equivalency),"% Some college, no degree",% A Associate's degree,...,% Graduate or professional degree,% high school graduate or higher,% bachelor's degree or higher,% White,% African American,% American Indian and Alaska Native,% Asian,% Native Hawaiian and Other Pacific Islander,% Some other race,cafe_count
0,1,3.412022,1.446834,0.0,0.0,17.2,16.3,29.0,16.7,7.5,...,3.2,66.6,13.2,10.4,32.0,0.8,1.5,0.1,52.8,3.0
1,2,3.175676,1.231801,0.0,0.0,18.7,18.6,26.2,18.1,5.6,...,4.4,62.7,12.8,11.6,34.5,0.4,1.3,0.1,48.8,2.0
2,2,3.477435,1.406593,0.0,0.0,18.2,20.5,25.8,17.8,5.6,...,2.6,61.3,12.1,13.4,28.5,0.2,1.6,0.0,53.6,1.0
3,1,3.774120,1.461673,2.0,3.0,16.2,15.5,29.4,18.4,6.6,...,4.1,68.3,13.9,19.3,20.5,0.4,3.1,0.1,52.2,15.0
4,1,3.546569,1.274436,0.0,1.0,14.0,14.0,29.1,17.0,6.8,...,6.0,72.0,19.0,20.0,35.7,0.7,6.3,0.0,33.3,8.0
5,1,3.358816,1.596905,1.0,4.0,16.8,15.4,28.7,16.8,5.4,...,4.6,67.8,16.9,14.1,21.1,0.7,3.5,0.0,56.2,4.0
6,1,3.835586,1.396667,0.0,0.0,17.1,14.2,25.6,17.6,6.4,...,6.6,68.7,19.0,10.7,38.8,0.5,1.8,0.2,44.1,2.0
7,1,3.193541,1.629340,0.0,0.0,18.7,17.0,26.5,16.9,7.8,...,2.9,64.3,13.0,11.1,31.7,0.2,1.0,0.0,52.0,2.0
8,2,3.466068,1.556452,0.0,0.0,19.0,17.3,25.9,18.9,5.5,...,3.1,63.7,13.4,8.6,40.7,0.9,1.3,0.0,45.7,3.0
9,2,3.658730,1.755556,0.0,3.0,23.1,20.3,26.9,15.5,5.3,...,2.2,56.6,9.0,15.6,29.1,0.8,0.7,0.0,50.9,3.0


In [257]:
y = dataset['gentrification_label']
# 2 is gentrifying, 1 is non-gentrifying and 0 is high income
x = dataset.drop('gentrification_label',axis = 1)

In [258]:
#y = pd.get_dummies(y,drop_first=True) for multi-nomial distrubted variables

In [259]:
#First randomly assigned y's and then found significant parameters

for value in range(len(y)):
    if y[value] == 0: #0 is High income
        y[value] = np.random.randint(1, 3)

In [260]:
x.head()

,avg_rating,avg_price,starbucks_count,bars,"%, Less than 9th grade","%, 9th to 12th grade, no diploma",% High school graduate (includes equivalency),"% Some college, no degree",% A Associate's degree,% Bachelor's degree,% Graduate or professional degree,% high school graduate or higher,% bachelor's degree or higher,% White,% African American,% American Indian and Alaska Native,% Asian,% Native Hawaiian and Other Pacific Islander,% Some other race,cafe_count
0,3.412022,1.446834,0.0,0.0,17.2,16.3,29.0,16.7,7.5,10.1,3.2,66.6,13.2,10.4,32.0,0.8,1.5,0.1,52.8,3.0
1,3.175676,1.231801,0.0,0.0,18.7,18.6,26.2,18.1,5.6,8.4,4.4,62.7,12.8,11.6,34.5,0.4,1.3,0.1,48.8,2.0
2,3.477435,1.406593,0.0,0.0,18.2,20.5,25.8,17.8,5.6,9.5,2.6,61.3,12.1,13.4,28.5,0.2,1.6,0.0,53.6,1.0
3,3.774120,1.461673,2.0,3.0,16.2,15.5,29.4,18.4,6.6,9.9,4.1,68.3,13.9,19.3,20.5,0.4,3.1,0.1,52.2,15.0
4,3.546569,1.274436,0.0,1.0,14.0,14.0,29.1,17.0,6.8,12.9,6.0,72.0,19.0,20.0,35.7,0.7,6.3,0.0,33.3,8.0


In [261]:
y.shape

(157,)

In [262]:
# Stratified Split of train and test data
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
sss.get_n_splits(x, y)

5

In [263]:
for train_index, test_index in sss.split(x, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
print(y_train.value_counts())
print(y_test.value_counts())

2    75
1    42
Name: gentrification_label, dtype: int64
2    25
1    15
Name: gentrification_label, dtype: int64


In [264]:
def model_performance_metrics(model, y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    if model.classes_[0] == 1:
        cm = np.array([[cm[1,1], cm[1,0]], [cm[0,1], cm[0,0]]])

    
    tp = cm[1,1]
    fn = cm[1,0]
    fp = cm[0,1]
    tn = cm[0,0]
    return_values = [cm, 
                     'Recall (TPR) = {0:.2%}'.format(tp/(tp+fn)),
                     'Fallout (FPR) = {0:.2%}'.format(fp/(fp+tn)),
                     'Accuracy (Acc) = {0:.2%}'.format((tp+tn)/(tp+tn+fp+fn))]
    return return_values

In [265]:
from sklearn import linear_model, metrics
reg = linear_model.LinearRegression()
reg.fit(x_train,y_train) #training the algorithm
y_pred = reg.predict(x_test)

In [266]:
import statsmodels.api as sm
from scipy import stats

#x2 = sm.add_constant(x)
est = sm.OLS(y, x)
est2 = est.fit()
print(est2.summary())

                             OLS Regression Results                             
Dep. Variable:     gentrification_label   R-squared:                       0.933
Model:                              OLS   Adj. R-squared:                  0.923
Method:                   Least Squares   F-statistic:                     95.51
Date:                  Sat, 27 Apr 2019   Prob (F-statistic):           3.31e-70
Time:                          19:28:58   Log-Likelihood:                -94.366
No. Observations:                   157   AIC:                             228.7
Df Residuals:                       137   BIC:                             289.9
Df Model:                            20                                         
Covariance Type:              nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [267]:
#Significant parameters
# avg_rating
# bachelor degree
# Graduate or professional degree 
# bachelor's degree or higher  

In [268]:
x

,avg_rating,avg_price,starbucks_count,bars,"%, Less than 9th grade","%, 9th to 12th grade, no diploma",% High school graduate (includes equivalency),"% Some college, no degree",% A Associate's degree,% Bachelor's degree,% Graduate or professional degree,% high school graduate or higher,% bachelor's degree or higher,% White,% African American,% American Indian and Alaska Native,% Asian,% Native Hawaiian and Other Pacific Islander,% Some other race,cafe_count
0,3.412022,1.446834,0.0,0.0,17.2,16.3,29.0,16.7,7.5,10.1,3.2,66.6,13.2,10.4,32.0,0.8,1.5,0.1,52.8,3.0
1,3.175676,1.231801,0.0,0.0,18.7,18.6,26.2,18.1,5.6,8.4,4.4,62.7,12.8,11.6,34.5,0.4,1.3,0.1,48.8,2.0
2,3.477435,1.406593,0.0,0.0,18.2,20.5,25.8,17.8,5.6,9.5,2.6,61.3,12.1,13.4,28.5,0.2,1.6,0.0,53.6,1.0
3,3.774120,1.461673,2.0,3.0,16.2,15.5,29.4,18.4,6.6,9.9,4.1,68.3,13.9,19.3,20.5,0.4,3.1,0.1,52.2,15.0
4,3.546569,1.274436,0.0,1.0,14.0,14.0,29.1,17.0,6.8,12.9,6.0,72.0,19.0,20.0,35.7,0.7,6.3,0.0,33.3,8.0
5,3.358816,1.596905,1.0,4.0,16.8,15.4,28.7,16.8,5.4,12.3,4.6,67.8,16.9,14.1,21.1,0.7,3.5,0.0,56.2,4.0
6,3.835586,1.396667,0.0,0.0,17.1,14.2,25.6,17.6,6.4,12.4,6.6,68.7,19.0,10.7,38.8,0.5,1.8,0.2,44.1,2.0
7,3.193541,1.629340,0.0,0.0,18.7,17.0,26.5,16.9,7.8,10.1,2.9,64.3,13.0,11.1,31.7,0.2,1.0,0.0,52.0,2.0
8,3.466068,1.556452,0.0,0.0,19.0,17.3,25.9,18.9,5.5,10.2,3.1,63.7,13.4,8.6,40.7,0.9,1.3,0.0,45.7,3.0
9,3.658730,1.755556,0.0,3.0,23.1,20.3,26.9,15.5,5.3,6.8,2.2,56.6,9.0,15.6,29.1,0.8,0.7,0.0,50.9,3.0


In [269]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
lrn = LogisticRegression()
lrn.fit(x_train, y_train)
y_pred = lrn.predict(x_test)

model_performance_metrics(lrn, y_test, y_pred)

[array([[22,  3],
        [13,  2]], dtype=int64),
 'Recall (TPR) = 13.33%',
 'Fallout (FPR) = 12.00%',
 'Accuracy (Acc) = 60.00%']

In [270]:
import statsmodels.discrete.discrete_model as sm
# for value in range(len(y)):
#     y[value] = y[value]-1
# y.unique()
model = LogisticRegression(fit_intercept = False, C = 1e9)
mdl = model.fit(x, y)
model.coef_


array([[-3.28304596e-01, -1.20224713e+00,  8.44856575e-02,
        -9.28049265e-04,  4.46138589e-01,  3.53775053e-01,
         5.87605572e+00,  6.20581046e+00,  6.14436344e+00,
         6.78957758e+00,  6.69253664e+00, -5.75337672e+00,
        -7.13530620e-01, -2.57849590e-01, -2.57065867e-01,
        -9.15524126e-01, -2.80710368e-01,  6.21097020e-01,
        -3.27719404e-01,  2.83525770e-02]])

In [271]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
model_performance_metrics(knn, y_test, y_pred)

[array([[22,  3],
        [12,  3]], dtype=int64),
 'Recall (TPR) = 20.00%',
 'Fallout (FPR) = 12.00%',
 'Accuracy (Acc) = 62.50%']

In [272]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
model_performance_metrics(clf, y_test, y_pred)

[array([[21,  4],
        [ 9,  6]], dtype=int64),
 'Recall (TPR) = 40.00%',
 'Fallout (FPR) = 16.00%',
 'Accuracy (Acc) = 67.50%']

In [273]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)

y_pred = lda.predict(x_test)
model_performance_metrics(lda, y_test, y_pred)

[array([[21,  4],
        [13,  2]], dtype=int64),
 'Recall (TPR) = 13.33%',
 'Fallout (FPR) = 16.00%',
 'Accuracy (Acc) = 57.50%']

In [274]:
#!pip install xgboost

In [275]:
import xgboost as xgb
gbm= xgb.XGBClassifier(max_depth=10, n_estimators=300, learning_rate=0.01).fit(x_train, y_train)

y_pred = gbm.predict(x_test)
model_performance_metrics(gbm, y_test, y_pred)

[array([[22,  3],
        [13,  2]], dtype=int64),
 'Recall (TPR) = 13.33%',
 'Fallout (FPR) = 12.00%',
 'Accuracy (Acc) = 60.00%']